# Analisi cyclictest dai log raw
Legge `cyclic_baseline.log` e `cyclic_stress.log` (formato `thread:iter:latency_us`) e costruisce grafici di latenza, deviazione standard e jitter.

## Setup
- Import delle librerie.
- Percorsi dei log (modifica se sono in un'altra cartella).

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

BASELINE_LOG = Path('cyclic_baseline.log')
STRESS_LOG   = Path('cyclic_stress.log')
%matplotlib inline


## Caricamento log raw
Legge righe `thread:iter:latency_us`, salta commenti, crea un DataFrame unico con colonna `scenario`.

In [ ]:
def load_cyclictest_log(path: Path, scenario_name: str) -> pd.DataFrame:
    recs = []
    with path.open() as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            try:
                t_str, i_str, lat_str = line.split(':')
                recs.append({
                    'scenario': scenario_name,
                    'thread': int(t_str),
                    'iter': int(i_str),
                    'latency_us': float(lat_str),
                })
            except ValueError:
                continue
    return pd.DataFrame.from_records(recs)


df_base = load_cyclictest_log(BASELINE_LOG, 'baseline')
df_stress = load_cyclictest_log(STRESS_LOG, 'stress')

df_all = pd.concat([df_base, df_stress], ignore_index=True)
print(df_all.head())
print('Campioni totali:', len(df_all))


## Statistiche per scenario
Calcola count, media, deviazione standard, min, max, p99.9. Jitter per scenario = max - min.

In [ ]:
def scenario_stats(df: pd.DataFrame) -> pd.DataFrame:
    def p999(x):
        return np.percentile(x, 99.9)
    grouped = (
        df.groupby('scenario')['latency_us']
        .agg(count='count', mean='mean', std='std', min='min', max='max', p99_9=p999)
        .reset_index()
    )
    grouped['jitter'] = grouped['max'] - grouped['min']
    return grouped

stats = scenario_stats(df_all)
print(stats)


## Statistiche per thread
Per ogni thread e scenario: count, mean, std, min, max, p99.9 e jitter (=max-min).

In [ ]:
def thread_stats(df: pd.DataFrame) -> pd.DataFrame:
    def p999(x):
        return np.percentile(x, 99.9)
    g = (
        df.groupby(['scenario','thread'])['latency_us']
        .agg(count='count', mean='mean', std='std', min='min', max='max', p99_9=p999)
        .reset_index()
        .sort_values(['scenario','thread'])
    )
    g['jitter'] = g['max'] - g['min']
    return g

thread_df = thread_stats(df_all)
thread_df.head()


## Grafico: distribuzione latenze
Istogrammi normalizzati per baseline vs stress; linee verticali su max (worst-case) e p99.9.

In [ ]:
colors = {'baseline':'#1f77b4','stress':'#d62728'}
fig, ax = plt.subplots(figsize=(12,6))
for scenario in ['baseline','stress']:
    s = df_all[df_all['scenario']==scenario]['latency_us']
    max_v = s.max(); p999 = np.percentile(s, 99.9)
    ax.hist(s, bins=80, density=True, alpha=0.45, color=colors[scenario], label=f"{scenario} (max={max_v:.0f}µs, p99.9={p999:.0f}µs)")
    ax.axvline(max_v, color=colors[scenario], linestyle='-', linewidth=1.6)
    ax.axvline(p999, color=colors[scenario], linestyle=':', linewidth=1.2)
ax.set_xlabel('Latenza (µs)')
ax.set_ylabel('Densità (norm)')
ax.set_title('Distribuzione latenze baseline vs stress')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()


## Grafico: jitter per scenario
Barre verticali con jitter = max - min per baseline e stress.

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
labels = stats['scenario']; jitters = stats['jitter']
colors = ['#1f77b4' if s=='baseline' else '#d62728' for s in labels]
bars = ax.bar(labels, jitters, color=colors, alpha=0.8)
ax.set_ylabel('Jitter (max - min) µs')
ax.set_title('Jitter per scenario')
for bar, val in zip(bars, jitters):
    ax.text(bar.get_x()+bar.get_width()/2, val+val*0.03+0.5, f"{val:.1f} µs", ha='center', va='bottom', fontsize=9)
ax.grid(True, axis='y', alpha=0.3)
plt.show()


## Grafico: jitter per thread (top 15 peggiori)
Barre orizzontali ordinate per jitter = max-min, per individuare i thread più variabili.

In [ ]:
worst_jitter = thread_df.sort_values('jitter', ascending=False).head(15)
fig, ax = plt.subplots(figsize=(10,6))
ax.barh(
    worst_jitter.apply(lambda r: f"{r['scenario']} T{r['thread']}", axis=1),
    worst_jitter['jitter'],
    color=['#1f77b4' if s=='baseline' else '#d62728' for s in worst_jitter['scenario']],
    alpha=0.85,
)
ax.invert_yaxis()
ax.set_xlabel('Jitter (µs)')
ax.set_title('Top 15 jitter per thread')
ax.grid(True, axis='x', alpha=0.3)
plt.show()


## Boxplot latenze per scenario
Per confrontare mediana, quartili e outlier tra baseline e stress.

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
data_to_plot = [df_all[df_all['scenario']==s]['latency_us'] for s in ['baseline','stress']]
ax.boxplot(data_to_plot, labels=['baseline','stress'], showfliers=True, patch_artist=True)
ax.set_ylabel('Latenza (µs)')
ax.set_title('Boxplot latenze')
ax.grid(True, axis='y', alpha=0.3)
plt.show()


## Tabella finale per thread (con jitter)
Lista completa dei thread con jitter calcolato (min-max).

In [ ]:
thread_df
